## A simple swarm

In [8]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from core.agent import *
from core.world import * 
from core.map import *
import numpy as np
from core.render import * 

In [10]:
def initialize_swarm(world : World):
    swarm = [Agent() for i in range(1000)]
    for agent in swarm:
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    

In [11]:
world = World(dims = (100, 100),
              swarm_initialzier= initialize_swarm,
              resource_generator= RandomMapGenerator(),
              energy_model=EnergyModel()
              )
world.reset()

In [12]:
# Test movements by doing random actions
def update():
    world.update()
    for agent in world.agents:
        action_choice = np.random.randint(0, 4)
        choice = np.random.randint(1, 5)

        match (action_choice): 
            case 1: agent.move(choice)
            case 2: agent.pick_up(choice)
            case 3: agent.put_down(choice)

        # Communication protocol
        neighbors = agent.agents_in_range 

        # First receive messages
        received_messages = agent.get_messages()
        for msg in received_messages: 
            agent.add_relation(msg.sender, msg.message)
        agent.clear_messages()

        # Then send messages
        if len (neighbors) > 0:
            choice = np.random.choice(neighbors)

            tgt_agent = world.get_agent(choice)
            agent.talk_to(tgt_agent)

update()

SyntaxError: invalid syntax (2081275968.py, line 19)

In [6]:
render_world(world, (800, 800), update_fn=update, delay_s=0)

## Verify certain things about the simulation

In [7]:
swarm = world.agents

In [8]:
# Validate the observation space provided is sensible
obs = swarm[0].local_observation

print(obs.nearby_agents, obs.nearby_agents.shape)
print(obs.resource_types, obs.resource_types.shape)

[[  0   0   0 608   0   0   0]
 [  0   0   0 690   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0 639   0   0 881]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]] (7, 7)
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]] (7, 7)


In [9]:
# Validate that all swarm agents landed in different positions
positions = set()
for agent in swarm:
    pos = agent.current_position
    positions.add((pos[0], pos[1]))

print(len(positions), len(world.agents))

1000 1000


In [10]:
# Check for resources
rsrc_qty = world._resource_map._resource_quantity_map
qtys = []

for i in range(rsrc_qty.shape[0]):
    for j in range(rsrc_qty.shape[1]):
        if rsrc_qty[i, j] > 0:
            qtys.append(rsrc_qty[i, j])

print(qtys)

[2.6661249195381096, 1.070524693084025, 7.094339688523753, 4.261021753062482, 2.671801599231783, 5.700735871591871, 6.051354997181159, 3.385808873067144, 4.81234186949003, 3.599762037583342, 1.0, 5.9595266815444035, 6.228580488105543, 5.2500347515637475, 5.514146286418621, 9.096904439662973, 5.722918177067209, 0.5488003399033996, 8.369883749599948, 4.284563714029073, 3.039257799763491, 7.343699265797081, 8.294359570742053, 3.8536993501958117, 6.481847632713343, 3.781076518182384, 9.71157238605384, 2.7502559659312524, 5.148595511004993, 2.2927247173191145, 5.850026691964084, 7.302836651718959, 5.919631487134198, 7.04018326741029, 3.6459529348163286, 3.2741656014798175, 5.103347360794425, 4.96503229308065, 6.926195149433518, 6.0265991910645536, 1.7244716155386404, 2.5408050412812964, 5.7678525880511735, 2.523707324336756, 5.387441458036502, 2.089640113393349, 5.558906995136694, 2.07427792950921, 6.280659211432861, 6.255127700661143, 1.1190122991812688, 1.257202797985328, 4.98230289904565

In [11]:
# Check for agent state
state = swarm[0]._current_state.inventory
print(state)


[Resource(type=5, quantity=1.0), Resource(type=5, quantity=1.0), Resource(type=5, quantity=1.0), Resource(type=5, quantity=0.7436474509514754), Resource(type=5, quantity=0.949867430395785)]


# Perftest

In [34]:
import cProfile
def stress_test():

    for _ in range(0, 1000):
        update()

In [35]:
world.reset()
cProfile.run('stress_test()', sort = 'time')

         35014023 function calls in 11.761 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1000000    1.572    0.000    2.752    0.000 world.py:126(_get_nearby_agents)
  1000000    1.552    0.000    2.707    0.000 world.py:146(_get_nearby_resources)
     1000    1.433    0.001    2.673    0.003 world.py:85(_update_movement)
  6000000    1.264    0.000    1.264    0.000 {method 'copy' of 'numpy.ndarray' objects}
     1000    1.221    0.001   11.740    0.012 world.py:48(update)
  6000000    0.900    0.000    2.164    0.000 agent.py:205(current_position)
  2000000    0.621    0.000    0.621    0.000 {built-in method numpy.array}
  5000000    0.560    0.000    0.560    0.000 {built-in method builtins.max}
     1000    0.487    0.000    0.895    0.001 world.py:193(get_presence_mask)
  4000000    0.458    0.000    0.458    0.000 {built-in method builtins.min}
  1000000    0.440    0.000    0.590    0.000 models.py:11(forward)
 